# LangChain Experiement Number 1

In [6]:
import openai
import os
import dotenv
dotenv.load_dotenv()
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

## Sample Project

In [7]:
project_details="""
Basically you would put your hackathon idea in a text box
and we would run like 3 llm chains in the background
One for frontend one for backend, and then feed both chains into the last one which evaluates the two first chains if its feasible or not, and if it isn't, calls the first two chains again
The first two will also return design specifications, eg an outline of the frontend and backend
maybe another textbox in the frontend for skills/target categories
"""
project_technologies="""
OpenAI API
StreamLit
"""

### Setting Up Chain No. 1: Feature Retrieval

In [8]:
llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo-0613")
prompt = PromptTemplate(
    input_variables=["project_details", "project_technologies"],
    template="""
    Given the following project description and tech stack, identify and elaborate on the key frontend features that would be necessary for development. The frontend features should not involve backend api calls or database interactions. The frontend features should be described in terms of user stories or detailed feature requirements.
    This project is a hackathon project. Break apart the features into MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
    Project description: {project_details}
    Technologies: {project_technologies}
    """
)

chain = LLMChain(llm=llm, prompt=prompt)
frontend_features = chain.run({
    'project_details': project_details,
    'project_technologies': project_technologies
})
print(frontend_features)

MVP Frontend Features:
1. User Input: The frontend should have a text box where users can input their hackathon idea.
2. Display Design Specifications: The frontend should display the design specifications generated by the first chain, which outlines the frontend and backend components.
3. Feasibility Evaluation: The frontend should display the evaluation result from the last chain, indicating whether the idea is feasible or not.
4. Skills/Target Categories Input: The frontend should have another text box where users can input their skills or target categories.

Additional Frontend Features:
1. Real-time Feedback: As the user types their hackathon idea, the frontend should provide real-time feedback or suggestions to improve the idea.
2. Design Outline Visualization: The frontend can visually represent the design outline generated by the first chain, such as using diagrams or wireframes.
3. Feasibility Status Updates: The frontend can provide real-time updates on the feasibility evalua

## Setting Up Chain Two:  Feature Extraction

In [9]:
specification_prompt = PromptTemplate(
    input_variables=["frontend_features", "project_technologies"],
    template="""
    Given the extracted frontend features, create a detailed technical specification. 
    This specification should include the technologies to be used, the architecture, pages to be developed, and the components required for each page.
    However, they should be split into two categories: MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
    You should ignore the technologies for the backend and focus on the frontend.
    Please also mention any other technical considerations.

    Frontend Features: {frontend_features}
    Project Technologies: {project_technologies}
    """
)

specification_chain = LLMChain(llm=llm, prompt=specification_prompt)
specification = specification_chain.run({
    'frontend_features': frontend_features,
    'project_technologies': project_technologies
})
print(specification)

Technical Specification:

MVP Frontend Features:
1. User Input:
   - Technology: HTML, CSS, JavaScript
   - Component: Text box for users to input their hackathon idea

2. Display Design Specifications:
   - Technology: HTML, CSS, JavaScript
   - Component: Display area to show the design specifications generated by the first chain

3. Feasibility Evaluation:
   - Technology: HTML, CSS, JavaScript
   - Component: Display area to show the evaluation result from the last chain, indicating feasibility

4. Skills/Target Categories Input:
   - Technology: HTML, CSS, JavaScript
   - Component: Text box for users to input their skills or target categories

Additional Frontend Features:
1. Real-time Feedback:
   - Technology: HTML, CSS, JavaScript
   - Component: Real-time feedback or suggestions displayed as the user types their hackathon idea

2. Design Outline Visualization:
   - Technology: HTML, CSS, JavaScript
   - Component: Visual representation of the design outline generated by the f

## Setting Up Chain Three: AI Approval

In [10]:
approval_prompt = PromptTemplate(
    input_variables=["technical_specification", "aspect", "group_size", "group_experience"],
    template="""
    Given the developed technical specification, conduct a thorough review of the MVP Features only for any inconsistencies or issues. 
    Also, evaluate whether the MVP Features, can be realistically completed within the two day hackathon for {group_size} people.
    
    The MVP Features are specifically listed under the heading 'MVP Features'. 
    Please completely disregard any features or sections listed under 'Additional Features' or any similar headers.
    This specification is only for the {aspect} aspect of the project, and should not be evaluated for other aspects.

    Answer this question: Can the MVP Features be realistically completed within the two day hackathon for {group_size} people with this skill level: {group_experience}?
    Output only a json with keys 'approval' and 'comments'. 
    If yes, the value of 'approval' should be '1' and the value of 'comments' should be an empty string
    If not, the value of 'approval' should be '0' and the value of 'comments' should be a string with the issues and inconsistencies listed.

    Technical Specification: {technical_specification}
    
    Output only a json with keys 'approval' and 'comments'. 
    """
)

approval_chain = LLMChain(llm=llm, prompt=approval_prompt)
approval = approval_chain.run({
    'technical_specification': specification,
    'aspect': 'frontend',
    'group_size': '4',
    'group_experience': 'experienced'
})

print(approval)

{
  "approval": "1",
  "comments": ""
}


## Setting Up Chain Four: AI Refinement

In [11]:
refinement_prompt = PromptTemplate(
    input_variables=["project_details", "frontend_features", "project_technologies", "technical_specification", "aspect", "group_size", "approval"],
    template="""
    Refine the project based on the following information:
    Project Description: {project_details}
    Frontend Features: {frontend_features}
    Project Technologies: {project_technologies}
    Technical Specification: {technical_specification}
    Aspect: {aspect}
    Group Size: {group_size}
    Approval Status: {approval}
    If approval is '0', update frontend features and technical specification to meet the criteria, ignoring backend and database. Otherwise just return Approved!
    """
)
advanced_llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo-0613")
refinement_chain = LLMChain(llm=advanced_llm, prompt=refinement_prompt)
refinement = refinement_chain.run({
    'project_details': project_details,
    'frontend_features': frontend_features,
    'project_technologies': project_technologies,
    'technical_specification': specification,
    'aspect': 'frontend',
    'group_size': '4',
    'approval': approval
})
print(refinement)


Approved!
